In [1]:
# from quantdev.data import Databank
from quantdev.backtest import *
from quantdev.analysis import *
# from quantdev.trade import PortfolioManager, Position, Portfolio

# from typing import Literal
# import pandas as pd
# import numpy as np

In [2]:
roe = get_factor('roe')
pbr = get_factor('股價淨值比', asc=False)
mtm = get_factor('mtm_3m')
roe_strategy = backtesting(roe>=0.99, 'QR')
pbr_strategy = backtesting(pbr>=0.95, 'QR')
mtm_strategy = backtesting(mtm>=0.95, 'QR')

,Strategy,0050.TT
Annual return,12.44%,10.83%
Total return,976.40%,733.53%
Max drawdown,-53.65%,-55.66%
Annual volatility,17.42%,19.20%
Sharpe ratio,0.71,0.56
Calmar ratio,0.23,0.19
beta,0.42,-


,Strategy,0050.TT
Annual return,16.50%,10.83%
Total return,2150.49%,733.53%
Max drawdown,-66.87%,-55.66%
Annual volatility,16.07%,19.20%
Sharpe ratio,1.0,0.56
Calmar ratio,0.25,0.19
beta,0.51,-


,Strategy,0050.TT
Annual return,15.33%,10.83%
Total return,1698.89%,733.53%
Max drawdown,-66.14%,-55.66%
Annual volatility,21.93%,19.20%
Sharpe ratio,0.7,0.56
Calmar ratio,0.23,0.19
beta,0.67,-


In [4]:
strategies = {
    'roe': (roe_strategy, 1),
    'pbr': (pbr_strategy, 1),
    'mtm': (mtm_strategy, 1),
}
metastrategy = meta_backtesting(strategies, None)

,Meta strategy,roe,pbr,mtm,0050.TT
Annual return,15.29%,12.44%,16.50%,15.33%,10.83%
Total return,1686.11%,976.40%,2150.49%,1698.89%,733.53%
Max drawdown,-62.64%,-53.65%,-66.87%,-66.14%,-55.66%
Annual volatility,16.35%,17.42%,16.07%,21.93%,19.20%
Sharpe ratio,0.94,0.71,1.0,0.7,0.56
Calmar ratio,0.24,0.23,0.25,0.23,0.19
beta,0.55,0.42,0.51,0.67,-


In [5]:
metastrategy.report

BokehModel(combine_events=True, render_bundle={'docs_json': {'4f874f72-001b-4244-8990-6df7f09ef924': {'version…

In [226]:
import plotly.graph_objects as go

def plot_efficiency_frontier(returns:pd.DataFrame):
    

    fig = make_subplots(rows=2, cols=1, vertical_spacing=0.05)

    results = calc_random_portfolios(returns)
    max_sharpe = results.loc[results['sharpe'].argmax()]
    min_vol = results.loc[results['std_dev'].argmin()]
    
    fig.add_trace(go.Scatter(
        x=results['std_dev'],
        y=results['return'],
        mode='markers',
        marker=dict(
            size=5,
            color=results['return'],
            colorscale='PuBu',
            # showscale=True,
            opacity=0.3
        ),
        name='Weights',
        text=results['weights'].astype(str),
        showlegend=False,
    ), row=1, col=1)

    # Add maximum Sharpe ratio point
    fig.add_trace(go.Scatter(
        x=[max_sharpe['std_dev']],
        y=[max_sharpe['return']],
        mode='markers',
        marker=dict(size=10, symbol='diamond'),
        name=f"Max Sharpe: {str(max_sharpe['weights'])}",
        text=[str(max_sharpe['weights'])],
        showlegend=True,
    ), row=1, col=1)

    # Add minimum volatility point
    fig.add_trace(go.Scatter(
        x=[min_vol['std_dev']],
        y=[min_vol['return']],
        mode='markers',
        marker=dict(size=10, symbol='diamond'),
        name=f"Min volatility: {str(min_vol['weights'])}",
        text=[str(min_vol['weights'])],
        showlegend=True
    ), row=1, col=1)
    
    corr_plot = returns.corr()
    corr_plot = corr_plot\
        .mask(np.tril(np.ones(corr_plot.shape)).astype(bool))

    fig.add_trace(go.Heatmap(
        z=corr_plot.values,
        x=corr_plot.columns,
        y=corr_plot.index,
        colorscale='RdBu_r',
        text=corr_plot.map("{:.2f}".format).replace('nan', '').values,
        texttemplate="%{text}",
        showscale=False,
        # hoverongaps=False,
        # showgaps=False,
        # zmin=-1,
        # zmax=1,
        # colorbar=dict(thickness=20),
        
    ), row=2, col=1)
    
    fig.update_yaxes(tickformat=".0%", row=1, col=1)
    fig.update_xaxes(tickformat=".0%", row=1, col=1)
    
    # position
    fig.update_yaxes(domain=[0.0, 1.0], row=1, col=1)
    fig.update_yaxes(domain=[0.15, 0.45], row=2, col=1)
    fig.update_xaxes(domain=[0.0, 1.0], row=1, col=1)
    fig.update_xaxes(domain=[0.7, 1.0], row=2, col=1)

    # title
    fig.add_annotation(text=(f'Efficienct frontier'), x=0, y=1, yshift=40, xref="x domain", yref="y domain", showarrow=False, row=1, col=1)
    fig.add_annotation(text=(f'Corr'), x=0.1, y=1, yshift=20, xref="x domain", yref="y domain", showarrow=False, row=2, col=1)

    # Update layout
    fig.update_layout(
        legend=dict(x=0.01, y=1, xanchor='left', yanchor='top', bgcolor='rgba(0,0,0,0)'),
        xaxis_title='Annualised volatility',
        yaxis_title='Annualised returns',
        width=PlotMaster().fig_param['size']['w'],
        height=PlotMaster().fig_param['size']['h'],
        template=PlotMaster().fig_param['template'],
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        xaxis2=dict(showgrid=False),
        yaxis2=dict(showgrid=False),
    )

    return fig
plot_efficiency_frontier(returns)